In [1]:
import numpy as np 
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Embedding, Input,GRU
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text, sequence

from keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import numpy as np

Using Theano backend.


In [2]:
path = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/toxic/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
print(train.shape, test.shape)

(95851, 8) (226998, 2)


In [3]:
train['comment_text'].head(1)

0    Nonsense?  kiss off, geek. what I said is true...
Name: comment_text, dtype: object

In [4]:
# X_t[0]

In [5]:
list_sentences_train = train["comment_text"].fillna("unknown").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("unknown").values

In [6]:
max_features = 20000
maxlen = 100

In [7]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))

In [8]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)

list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [12]:
list_sentences_train[:4]

array([ "Nonsense?  kiss off, geek. what I said is true.  I'll have your account terminated.",
       '"\n\n Please do not vandalize pages, as you did with this edit to W. S. Merwin. If you continue to do so, you will be blocked from editing.    "',
       '"\n\n ""Points of interest"" \n\nI removed the ""points of interest"" section you added because it seemed kind of spammy. I know you probably didn\'t mean to disobey the rules, but generally, a point of interest tends to be rather touristy, and quite irrelevant to an area culture. That\'s just my opinion, though.\n\nIf you want to reply, just put your reply here and add {{talkback|Jamiegraham08}} on my talkpage.   "',
       "Asking some his nationality is a Racial offence. Wow wasn't aware of it.  Blocking me has shown your support towards your community. Thanku for that"], dtype=object)

In [13]:
list_tokenized_train[:4]

[[813, 4855, 200, 6773, 40, 7, 162, 8, 393, 234, 19, 20, 350, 14246],
 [46,
  33,
  14,
  673,
  114,
  17,
  6,
  87,
  21,
  13,
  71,
  2,
  917,
  366,
  22,
  6,
  283,
  2,
  33,
  38,
  6,
  44,
  16,
  170,
  31,
  121],
 [732,
  3,
  504,
  7,
  184,
  1,
  732,
  3,
  504,
  125,
  6,
  180,
  68,
  11,
  1682,
  449,
  3,
  13293,
  7,
  69,
  6,
  300,
  239,
  282,
  2,
  1,
  400,
  26,
  711,
  5,
  148,
  3,
  504,
  5369,
  2,
  16,
  270,
  4,
  378,
  1087,
  2,
  30,
  740,
  900,
  210,
  50,
  29,
  320,
  244,
  22,
  6,
  108,
  2,
  676,
  50,
  206,
  20,
  676,
  64,
  4,
  150,
  4707,
  15,
  29,
  1935],
 [896,
  61,
  66,
  2481,
  8,
  5,
  2547,
  4277,
  1414,
  594,
  709,
  3,
  11,
  836,
  35,
  43,
  1172,
  20,
  358,
  1037,
  20,
  486,
  12,
  9]]

In [54]:
def cnn_rnn():
    embed_size = 256
    inp = Input(shape=(maxlen, ))
    main = Embedding(max_features, embed_size)(inp)
    main = Dropout(0.2)(main)
#     main = Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(main)
#     main = MaxPooling1D(pool_size=2)(main)
#     main = Conv1D(filters=32, kernel_size=2, padding='same', activation='relu')(main)
#     main = MaxPooling1D(pool_size=2)(main)
    main = GRU(32)(main)
#     main = Dense(16, activation="relu")(main)
    main = Dense(6, activation="sigmoid")(main)
    model = Model(inputs=inp, outputs=main)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
    return model

In [57]:
from sklearn.model_selection import train_test_split

print('Positive Labels ')
any_category_positive = np.sum(y,1)
print(pd.value_counts(any_category_positive))

X_t_train, X_t_test, y_train, y_test = train_test_split(X_t, y,test_size = 0.20)
print('Training:', X_t_train.shape)
print('Testing:', X_t_test.shape)

Positive Labels 
0    86061
1     3833
3     2523
2     2107
4     1076
5      231
6       20
dtype: int64
Training: (76680, 100)
Testing: (19171, 100)


In [58]:
model = cnn_rnn()

file_path="model_best.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early

In [59]:
model.fit(X_t_train, y_train,
          validation_data=(X_t_test, y_test),
          batch_size=50,
          epochs=20,
          shuffle = True,
          callbacks=callbacks_list,
          verbose=2)
model.save('Whole_model.h5')

Train on 76680 samples, validate on 19171 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.05307, saving model to model_best.h5
272s - loss: 0.0793 - acc: 0.9762 - val_loss: 0.0531 - val_acc: 0.9808
Epoch 2/20
Epoch 00001: val_loss improved from 0.05307 to 0.05071, saving model to model_best.h5
305s - loss: 0.0456 - acc: 0.9834 - val_loss: 0.0507 - val_acc: 0.9817
Epoch 3/20
Epoch 00002: val_loss did not improve
307s - loss: 0.0370 - acc: 0.9861 - val_loss: 0.0518 - val_acc: 0.9817
Epoch 4/20
Epoch 00003: val_loss did not improve
313s - loss: 0.0293 - acc: 0.9890 - val_loss: 0.0589 - val_acc: 0.9811
Epoch 5/20
Epoch 00004: val_loss did not improve
306s - loss: 0.0228 - acc: 0.9917 - val_loss: 0.0633 - val_acc: 0.9804
Epoch 6/20
Epoch 00005: val_loss did not improve
306s - loss: 0.0179 - acc: 0.9935 - val_loss: 0.0711 - val_acc: 0.9797
Epoch 7/20
Epoch 00006: val_loss did not improve
306s - loss: 0.0137 - acc: 0.9952 - val_loss: 0.0824 - val_acc: 0.9791
Epoch 8/20
Epoch 0

KeyboardInterrupt: 

In [60]:
# model.load_weights(file_path)
y_test = model.predict(X_te)
sample_submission = pd.read_csv(path + "sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv(path + "Keras_embedding_layer1.csv", index=False)